In [1]:
from kubernetes.client import V1ObjectMeta
import kubeflow.katib as kc
from kubeflow.katib import constants
from kubeflow.katib import utils
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSpec
from kubeflow.katib import V1alpha3CollectorSpec
from kubeflow.katib import V1alpha3EarlyStoppingSetting
from kubeflow.katib import V1alpha3EarlyStoppingSpec
from kubeflow.katib import V1alpha3Experiment
from kubeflow.katib import V1alpha3ExperimentCondition
from kubeflow.katib import V1alpha3ExperimentList
from kubeflow.katib import V1alpha3ExperimentSpec
from kubeflow.katib import V1alpha3ExperimentStatus
from kubeflow.katib import V1alpha3FeasibleSpace
from kubeflow.katib import V1alpha3FileSystemPath
from kubeflow.katib import V1alpha3FilterSpec
from kubeflow.katib import V1alpha3GoTemplate
from kubeflow.katib import V1alpha3GraphConfig
from kubeflow.katib import V1alpha3Metric
from kubeflow.katib import V1alpha3MetricsCollectorSpec
from kubeflow.katib import V1alpha3NasConfig
from kubeflow.katib import V1alpha3ObjectiveSpec
from kubeflow.katib import V1alpha3Observation
from kubeflow.katib import V1alpha3Operation
from kubeflow.katib import V1alpha3OptimalTrial
from kubeflow.katib import V1alpha3ParameterAssignment
from kubeflow.katib import V1alpha3ParameterSpec
from kubeflow.katib import V1alpha3SourceSpec
from kubeflow.katib import V1alpha3Suggestion
from kubeflow.katib import V1alpha3SuggestionCondition
from kubeflow.katib import V1alpha3SuggestionList
from kubeflow.katib import V1alpha3SuggestionSpec
from kubeflow.katib import V1alpha3SuggestionStatus
from kubeflow.katib import V1alpha3TemplateSpec
from kubeflow.katib import V1alpha3Trial
from kubeflow.katib import V1alpha3TrialAssignment
from kubeflow.katib import V1alpha3TrialCondition
from kubeflow.katib import V1alpha3TrialList
from kubeflow.katib import V1alpha3TrialSpec
from kubeflow.katib import V1alpha3TrialStatus
from kubeflow.katib import V1alpha3TrialTemplate

In [2]:
algorithmsettings = V1alpha3AlgorithmSetting(
    name= None,
    value = None
)

algorithm = V1alpha3AlgorithmSpec(
    algorithm_name = "bayesianoptimization",
    algorithm_settings = [algorithmsettings]
)

# Metric Collector
collector = V1alpha3CollectorSpec(kind = "StdOut")
metrics_collector_spec = V1alpha3MetricsCollectorSpec(
    collector = collector
)

# Objective
objective = V1alpha3ObjectiveSpec(
    goal = 0.6,
    objective_metric_name = "val_accuracy",
    type = "maximize"
)

# Parameters
feasible_space = V1alpha3FeasibleSpace(min = "1", max = "2")
epochs_parameter_spec = V1alpha3ParameterSpec(
    feasible_space = feasible_space, 
    name = "--epochs",
    parameter_type = "int"
)

feasible_space = V1alpha3FeasibleSpace(min = "0.01", max = "0.05")
lr_parameter_spec = V1alpha3ParameterSpec(
    feasible_space = feasible_space, 
    name = "--learning_rate",
    parameter_type = "double"
)

parameters = [epochs_parameter_spec, lr_parameter_spec]

# Trialtemplate
go_template = V1alpha3GoTemplate(
    raw_template =   "apiVersion: \"kubeflow.org/v1\"\nkind: TFJob\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n tfReplicaSpecs:\n  Worker:\n    replicas: 1\n    restartPolicy: OnFailure\n    template:\n      spec:\n        containers:\n          - name: tensorflow\n            image: gcr.io/gsoc-kf-example/tf_2_text_classification:1.4\n            imagePullPolicy: Always\n            command:\n              - \"python3\"\n              - \"/app/text_classification_rnn.py\"\n              {{- with .HyperParameters}}\n              {{- range .}}\n              - \"{{.Name}}={{.Value}}\"\n              {{- end}}\n              {{- end}}"
)


trial_template= V1alpha3TrialTemplate(go_template=go_template)


# Experiment
experiment = V1alpha3Experiment(
    api_version="kubeflow.org/v1alpha3",
    kind="Experiment",
    metadata=V1ObjectMeta(name="text-classification-example",namespace="kubeflow-mailsforyashj"),

    spec=V1alpha3ExperimentSpec(
         algorithm = algorithm,
         max_failed_trial_count=3,
         max_trial_count=12,
         metrics_collector_spec= metrics_collector_spec ,
         objective = objective,
         parallel_trial_count=4,
         parameters = parameters ,
         trial_template = trial_template
    )
)

In [3]:
namespace = kc.utils.get_default_target_namespace()
kclient = kc.KatibClient()

### Create Experiment

In [4]:
kclient.create_experiment(experiment,namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-11T08:18:43Z',
  'generation': 1,
  'name': 'text-classification-example',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '30404428',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/text-classification-example',
  'uid': '227678f7-c34f-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.6,
   'objectiveMetricName': 'val_accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '2', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
   {'feasibleSpace': {'max': '0.05', 'min': '0.01'},
    'name': '--learning_rate',
    'parameterType': 'double'}],
  'trialTempla

### Get Single Experiment

In [5]:
kclient.get_experiment(name="text-classification-example", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-11T08:18:43Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 1,
  'name': 'text-classification-example',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '30408032',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/text-classification-example',
  'uid': '227678f7-c34f-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.6,
   'objectiveMetricName': 'val_accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '2', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
   {'feasibleSpace': {'max': '0.05', 'min': '0.01'},
    'name': '--learning_rate',


### Get All Experiments 

In [6]:
kclient.get_experiment(namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'items': [{'apiVersion': 'kubeflow.org/v1alpha3',
   'kind': 'Experiment',
   'metadata': {'creationTimestamp': '2020-07-11T08:18:43Z',
    'finalizers': ['update-prometheus-metrics'],
    'generation': 1,
    'name': 'text-classification-example',
    'namespace': 'kubeflow-mailsforyashj',
    'resourceVersion': '30408032',
    'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/text-classification-example',
    'uid': '227678f7-c34f-11ea-89e4-42010a8c0018'},
   'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
     'algorithmSettings': [{}]},
    'maxFailedTrialCount': 3,
    'maxTrialCount': 12,
    'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
    'objective': {'goal': 0.6,
     'objectiveMetricName': 'val_accuracy',
     'type': 'maximize'},
    'parallelTrialCount': 4,
    'parameters': [{'feasibleSpace': {'max': '2', 'min': '1'},
      'name': '--epochs',
      'parameterType':

### Get Experiment Status

In [7]:
kclient.get_experiment_status(name="text-classification-example", namespace=namespace)

'Succeeded'

### Is Experiment Succeded

In [8]:
kclient.is_experiment_succeeded(name="text-classification-example", namespace=namespace)

True

### List Trials of an Experiment

In [9]:
kclient.list_trials(name="text-classification-example", namespace=namespace)

[{'name': 'text-classification-example-5g8wrt8q', 'status': 'Succeeded'},
 {'name': 'text-classification-example-82pqdz9h', 'status': 'Succeeded'},
 {'name': 'text-classification-example-hqbntrnm', 'status': 'Succeeded'},
 {'name': 'text-classification-example-lkpxtj8p', 'status': 'Succeeded'},
 {'name': 'text-classification-example-n76lsmbc', 'status': 'Succeeded'},
 {'name': 'text-classification-example-rxstj6xb', 'status': 'Succeeded'},
 {'name': 'text-classification-example-z4gk4ql7', 'status': 'Succeeded'}]

### List All Experiments

In [10]:
kclient.list_experiments(namespace=namespace)

[{'name': 'text-classification-example', 'status': 'Succeeded'}]

### Get Optimal Hyperparameter

In [11]:
kclient.get_optimal_hyperparmeters(name="text-classification-example", namespace=namespace)

{'currentOptimalTrial': {'bestTrialName': 'text-classification-example-z4gk4ql7',
  'observation': {'metrics': [{'name': 'val_accuracy', 'value': 0.61}]},
  'parameterAssignments': [{'name': '--epochs', 'value': '2'},
   {'name': '--learning_rate', 'value': '0.011231537311831038'}]}}

### Delete Experiment

In [12]:
kclient.delete_experiment(name="text-classification-example", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-07-11T08:18:43Z',
  'deletionGracePeriodSeconds': 0,
  'deletionTimestamp': '2020-07-11T14:06:33Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 2,
  'name': 'text-classification-example',
  'namespace': 'kubeflow-mailsforyashj',
  'resourceVersion': '30634815',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kubeflow-mailsforyashj/experiments/text-classification-example',
  'uid': '227678f7-c34f-11ea-89e4-42010a8c0018'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 12,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.6,
   'objectiveMetricName': 'val_accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 4,
  'parameters': [{'feasibleSpace': {'max': '2', 'min': '1'},
    'name': '--epochs',
    'parameterType': 'int'},
  